In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as dt

from dateutil.relativedelta import *
from dateutil.parser import *
from sqlalchemy.dialects.sqlite import \
            BLOB, BOOLEAN, CHAR, DATE, DATETIME, DECIMAL, FLOAT, \
            INTEGER, NUMERIC, JSON, SMALLINT, TEXT, TIME, TIMESTAMP, \
            VARCHAR

# Reflect Tables into SQLAlchemy ORM

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import sqlite3

# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite",echo=False)

# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare (engine, reflect = True)

# View all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [3]:
# Save references to each table
measurement = base.classes.measurement
station = base.classes.station

In [4]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [5]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [6]:
measurement_columns = inspector.get_columns('measurement')
for column in measurement_columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [7]:
station_columns = inspector.get_columns('station')
for column in station_columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [8]:
# Find the most recent date in the data set.
latest_date = session.query(measurement.date).order_by((measurement.date.desc())).first()
end_date=dt.datetime.strptime(latest_date[0],'%Y-%m-%d')
timestamp_end = dt.datetime.strftime(end_date,'%Y-%m-%d')


In [9]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
conn = engine.connect()
measurements_df = pd.read_sql_query('SELECT * from measurement',con=conn)

# Calculate the date one year from the last date in data set.
start_date = end_date + relativedelta(months=-12)
timestamp_start = dt.datetime.strftime(start_date,'%Y-%m-%d')

In [39]:
(timestamp_start,timestamp_end)

('2016-08-23', '2017-08-23')

In [54]:
# Perform a query to retrieve the data and precipitation scores

prcp_result = engine.execute('SELECT date,prcp FROM measurement WHERE date BETWEEN :start AND :end',(timestamp_start,timestamp_end)).fetchall()


# Save the query results as a Pandas DataFrame and set the index to the date column

prcp_df = pd.DataFrame(prcp_result)
prcp_final = prcp_df.rename(columns={0:'Date',1:'Precipitation'}).set_index('Date').dropna()

# Sort the dataframe by date
prcp_final.sort_values(by=['Date'])

# Use Pandas Plotting with Matplotlib to plot the data



,Precipitation
Date,
2016-08-23,0.00
2016-08-23,NaN
2016-08-23,1.79
2016-08-23,0.05
2016-08-23,0.15
...,...
2017-08-22,0.00
2017-08-23,0.00
2017-08-23,0.00


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()